In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [2]:
import matplotlib.pyplot as plt
#import matplotlib.ticker as mticker
import folium
from folium import Choropleth, Circle, Marker
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
%matplotlib inline


## Loading and Displaying Well Log Data from LAS


## Introduction
[Log ASCII standard (LAS)](https://en.wikipedia.org/wiki/Log_ASCII_standard#:~:text=Log%20ASCII%20standard%20(LAS)%20is,subsurface%20stratigraphy%20in%20a%20well) is a standard file format common in the oil-and-gas and water well industries to store well log information. Well logging is used to investigate and characterize the subsurface stratigraphy in a well.

Its structured file format that was developed in 1980 by the Canadian Well Logging Society and it is used to store log curve data as well as information about the well.

You can use various softwares to lookat the LAS data. To open and manupulate in python you can either write your own script or use available developed packages. Here, I will use 'lasio'.
More info on the library can be found at: https://lasio.readthedocs.io/en/latest/

In [1]:
import lasio


In [2]:
well = lasio.read("48_10b-_9_jwl_JWL_FILE_1682139.las")
well

In [3]:
well.sections.keys()

dict_keys(['Version', 'Well', 'Curves', 'Parameter', 'Other'])

In [4]:
well.sections['Well']

[HeaderItem(mnemonic="STRT", unit="F", value="181.1024", descr="Top Depth"),
 HeaderItem(mnemonic="STOP", unit="F", value="12631.2336", descr="Bottom Depth"),
 HeaderItem(mnemonic="STEP", unit="F", value="0.6562", descr="Depth Increment"),
 HeaderItem(mnemonic="NULL", unit="", value="-999.25", descr="Null Value"),
 HeaderItem(mnemonic="FLD", unit="", value="EXPLORATION", descr="Field Name"),
 HeaderItem(mnemonic="COMP", unit="", value="BP EX OPE C", descr="OPERATOR"),
 HeaderItem(mnemonic="LOC", unit="", value="", descr="FIELD LOCATION"),
 HeaderItem(mnemonic="CNTY", unit="", value="", descr="COUNTY"),
 HeaderItem(mnemonic="STAT", unit="", value="0", descr="STATE"),
 HeaderItem(mnemonic="CTRY", unit="", value="UK", descr="COUNTRY"),
 HeaderItem(mnemonic="SRVC", unit="", value="", descr="LOGGING COMPANY CODE"),
 HeaderItem(mnemonic="DATE", unit="", value="", descr="RUN DATE"),
 HeaderItem(mnemonic="WN", unit="", value="48/10B- 9", descr="NAME"),
 HeaderItem(mnemonic="WBN", unit="", valu

In [5]:
for i in well.sections['Well']:
    print(f"{i.descr} ({i.mnemonic},{i.unit}): \t\t {i.value}")

Top Depth (STRT,F): 		 181.1024
Bottom Depth (STOP,F): 		 12631.2336
Depth Increment (STEP,F): 		 0.6562
Null Value (NULL,): 		 -999.25
Field Name (FLD,): 		 EXPLORATION
OPERATOR (COMP,): 		 BP EX OPE C
FIELD LOCATION (LOC,): 		 
COUNTY (CNTY,): 		 
STATE (STAT,): 		 0
COUNTRY (CTRY,): 		 UK
LOGGING COMPANY CODE (SRVC,): 		 
RUN DATE (DATE,): 		 
NAME (WN,): 		 48/10B- 9
WELLBORE (WBN,): 		 48/10B- 9
COUNTRY (NATI,): 		 UK
BIT SIZE 1 (BS1,): 		 0.0
BIT SIZE 2 (BS2,): 		 0.0
CSG 1 DRILL DEP (CBD1,): 		 0.000 M
CSG 1 LOG DEP (CBL1,): 		 0.000 M
CSG 1 SIZE (CS1,): 		 0.0
CSG 1 WT (CW1,): 		 0.0
CSG 2 DRILL DEP (CBD2,): 		 0.000 M
CSG 2 LOG DEP (CBL2,): 		 0.000 M
CSG 2 SIZE (CS2,): 		 0.0
CSG 2 WT (CW2,): 		 0.0
RIG NAME (COUN,): 		 NULL
SPUD DATE (SPUD,): 		 1990/08/26
SURF. LATITUDE (LATI,): 		 53:43: 5.67984N
SURF. LONGITUDE (LONG,): 		 1:52:24.24000E
TD 1 DRILLER (BSD1,): 		 0.000 M
TD 2 DRILLER (BSD2,): 		 0.000 M
API NUMBER (ZZZZ,): 		 4177
UTM ZONE (UTZC,): 		 31
SURF. UTM E (UTME,

In [6]:
well.curves.keys()

['DEPT',
 'CALI',
 'ILM',
 'ILD',
 'DT',
 'TENS',
 'CGR',
 'DTL',
 'RHOB',
 'DRHO',
 'PEF',
 'NPHI',
 'THOR',
 'URAN',
 'GR',
 'POTA']

To see what curves are present within the las file, we can repeat the process with the Curve Item object and call upon the `unit` and `descr` functions to get info on the units and the curve's description.
The enumerate function allows us to keep a count of the number of curves that are present within the file. As enumerate returns a 0 on the first loop, we need to 1 to it if we want to include the depth curve.

In [7]:
for i, curve in enumerate(well.curves):
    print(f"Curve: {curve.mnemonic}, \t Units: {curve.unit}, \t Description: {curve.descr}")
print(f"There are: {i+1} curves in this file")

Curve: DEPT, 	 Units: F, 	 Description: 1  DEPTH
Curve: CALI, 	 Units: IN, 	 Description: 2  CALI
Curve: ILM, 	 Units: OHMM, 	 Description: 3  ILM
Curve: ILD, 	 Units: OHMM, 	 Description: 4  ILD
Curve: DT, 	 Units: US/F, 	 Description: 5  DT
Curve: TENS, 	 Units: LB, 	 Description: 6  TENS
Curve: CGR, 	 Units: GAPI, 	 Description: 7  CGR
Curve: DTL, 	 Units: US/F, 	 Description: 8  DTL
Curve: RHOB, 	 Units: G/C3, 	 Description: 9  RHOB
Curve: DRHO, 	 Units: G/C3, 	 Description: 10  DRHO
Curve: PEF, 	 Units: NULL, 	 Description: 11  PEF
Curve: NPHI, 	 Units: %, 	 Description: 12  NPHI
Curve: THOR, 	 Units: PPM, 	 Description: 13  THOR
Curve: URAN, 	 Units: PPM, 	 Description: 14  URAN
Curve: GR, 	 Units: GAPI, 	 Description: 15  GR
Curve: POTA, 	 Units: NULL, 	 Description: 16  POTA
There are: 16 curves in this file


### you can delete make changes and write the file in a new .las file

In [8]:
well.sections['Well']['CTRY'] = 'Norway'

## Converting LAS File to a Pandas Dataframe
Data loaded in using LASIO can be converted to a pandas dataframe using the .df() function. This allows us to easily plot data and pass it into one of the many machine learning algorithms.

In [9]:
df = well.df()
df

,CALI,ILM,ILD,DT,TENS,CGR,DTL,RHOB,DRHO,PEF,NPHI,THOR,URAN,GR,POTA
DEPT,,,,,,,,,,,,,,,
181.1024,NaN,NaN,NaN,NaN,771.2344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.5620,NaN
181.7585,NaN,NaN,NaN,NaN,771.1523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.6920,NaN
182.4147,NaN,NaN,NaN,NaN,771.0713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.7963,NaN
183.0709,NaN,NaN,NaN,NaN,770.9902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6101,NaN
183.7270,NaN,NaN,NaN,NaN,770.9111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.7897,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12628.6089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6924,0.0655,5.0506,NaN,NaN,NaN,NaN,NaN
12629.2651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7019,0.0671,5.2300,NaN,NaN,NaN,NaN,NaN
12629.9213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.7175,0.0775,5.5663,NaN,NaN,NaN,NaN,NaN
